# Classifier comparison

A comparison of a several classifiers on CICIDS2017 webattacks dataset.

Sources:

* CICIDS2017: https://www.unb.ca/cic/datasets/ids-2017.html
* Scikit-learn demo: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
* Overview of classification metrics: http://www.machinelearning.ru/wiki/images/d/de/Voron-ML-Quality-slides.pdf

## Reading and preparing data

Read undersampled (balanced) and preprocessed data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://github.com/fisher85/ml-cybersecurity/blob/master/python-web-attack-detection/datasets/web_attacks_balanced.zip?raw=true -O web_attacks_balanced.zip
!unzip -u web_attacks_balanced.zip
df = pd.read_csv('web_attacks_balanced.csv')

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


The "Label" column is encoded as follows: "BENIGN" = 0, attack = 1.

In [3]:
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
y = df['Label'].values

Select the features.

In [4]:
webattack_features = ['Average Packet Size', 'Flow Bytes/s', 'Max Packet Length', 'Packet Length Mean', 
                      'Fwd Packet Length Mean', 'Subflow Fwd Bytes', 'Fwd IAT Min', 'Avg Fwd Segment Size',
                      'Total Length of Fwd Packets', 'Fwd IAT Std', 'Fwd Packet Length Max', 'Flow IAT Mean',
                      'Fwd Header Length', 'Flow Duration', 'Flow Packets/s', 'Fwd IAT Mean',
                      'Fwd IAT Total', 'Fwd Packets/s', 'Flow IAT Std', 'Fwd IAT Max']

In [5]:
X = df[webattack_features]
print(X.shape, y.shape)

(7267, 20) (7267,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 3545, 1: 1541}

In [7]:
X.head()

,Average Packet Size,Flow Bytes/s,Max Packet Length,Packet Length Mean,Fwd Packet Length Mean,Subflow Fwd Bytes,Fwd IAT Min,Avg Fwd Segment Size,Total Length of Fwd Packets,Fwd IAT Std,Fwd Packet Length Max,Flow IAT Mean,Fwd Header Length,Flow Duration,Flow Packets/s,Fwd IAT Mean,Fwd IAT Total,Fwd Packets/s,Flow IAT Std,Fwd IAT Max
0,58.500000,7.835455e+03,66.0,46.800000,34.0,68.0,4.0,34.0,68.0,0.000000,34.0,8508.333333,64.0,25525.0,156.709109,4.0,4.0,78.354554,14730.803450,4.0
1,80.000000,1.346341e+06,94.0,64.000000,44.0,88.0,49.0,44.0,88.0,0.000000,44.0,68.333333,64.0,205.0,19512.195120,49.0,49.0,9756.097561,34.355980,49.0
2,63.500000,1.370370e+06,79.0,50.800000,32.0,64.0,1.0,32.0,64.0,0.000000,32.0,54.000000,64.0,162.0,24691.358020,1.0,1.0,12345.679010,91.798693,1.0
3,35.071429,1.239899e+06,163.0,32.733333,16.5,165.0,1.0,16.5,165.0,49.183331,79.0,30.461538,336.0,396.0,35353.535350,44.0,396.0,25252.525250,27.986948,132.0
4,102.500000,6.601925e+03,105.0,68.333333,50.0,50.0,0.0,50.0,50.0,0.000000,50.0,23478.000000,32.0,23478.0,85.186132,0.0,0.0,42.593066,0.000000,0.0


## Classifier comparison

The operation may take a long time, 3-5 minutes depending on the computer performance.

In [8]:
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('ABoost', AdaBoostClassifier()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('MLP', MLPClassifier()))

print('Model\tAcc\tPr\tRecall\tF1\tExecution')
      
for name, model in models:
    start_time = time.time()
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=24)    

    accuracy = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy').mean()
    precision = cross_val_score(model, X_train, y_train, cv=kfold, scoring='precision').mean()
    recall = cross_val_score(model, X_train, y_train, cv=kfold, scoring='recall').mean()
    f1_score = cross_val_score(model, X, y, cv=kfold, scoring='f1_weighted').mean()
    
    delta = time.time() - start_time
    print('{}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.2f} secs'.format(name, accuracy, precision, recall, f1_score, delta))

Model	Acc	Pr	Recall	F1	Execution
KNN	0.968	0.939	0.957	0.967	5.20 secs
SVM	0.700	0.629	0.030	0.594	92.75 secs
CART	0.974	0.988	0.924	0.972	1.17 secs
RF	0.975	0.987	0.933	0.969	1.22 secs
ABoost	0.978	0.971	0.956	0.974	18.05 secs
LR	0.954	0.950	0.896	0.948	6.04 secs
NB	0.729	0.529	0.958	0.730	0.28 secs
LDA	0.941	0.919	0.882	0.937	0.78 secs
QDA	0.858	0.772	0.548	0.948	0.98 secs
MLP	0.907	0.914	0.950	0.953	25.49 secs
